In [3]:
import joblib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Load encoders, scaler, and model
team_encoder = joblib.load("../model/team_encoder.pkl")
opponent_encoder = joblib.load("../model/opponent_encoder.pkl")
scaler = joblib.load("../model/scaler.pkl")
model = joblib.load("../model/nba_score_model.pkl")

# Load list of team names
team_names = list(team_encoder.classes_)

# Style title
display(HTML("<h2 style='color: #1d3557; font-family: Arial, sans-serif; text-align: center;'>🏀 NBA Score Prediction</h2>"))

# Create dropdown widgets for team selection with enhanced style
team_widget = widgets.Dropdown(
    options=team_names,
    description="Your Team:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%'),
    tooltip="Select the team you are predicting for"
)

opponent_widget = widgets.Dropdown(
    options=team_names,
    description="Opponent Team:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%'),
    tooltip="Select the opponent team"
)

# Create sliders for statistical input with more space and formatting
fg_pct_widget = widgets.FloatSlider(
    value=0.45, min=0.3, max=0.6, step=0.01,
    description="FG%:", readout_format=".2f",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

ft_pct_widget = widgets.FloatSlider(
    value=0.75, min=0.5, max=1.0, step=0.01,
    description="FT%:", readout_format=".2f",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

fg3_pct_widget = widgets.FloatSlider(
    value=0.35, min=0.2, max=0.5, step=0.01,
    description="3PT%:", readout_format=".2f",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

ast_widget = widgets.FloatSlider(
    value=25, min=5, max=40, step=1,
    description="AST:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

reb_widget = widgets.FloatSlider(
    value=40, min=20, max=60, step=1,
    description="REB:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

# Create prediction button with custom styling
predict_button = widgets.Button(
    description="Predict Score",
    button_style="success",
    layout=widgets.Layout(width='50%', height='40px'),
    style={'font_weight': 'bold'}
)

output = widgets.Output()

# Define prediction logic
def predict_score(b):
    with output:
        clear_output()

        # Encode teams (your team and opponent)
        team_encoded = team_encoder.transform([team_widget.value])[0]
        opponent_encoded = opponent_encoder.transform([opponent_widget.value])[0]

        # Collect feature values
        input_data = np.array([[
            team_encoded,
            opponent_encoded,
            fg_pct_widget.value,
            ft_pct_widget.value,
            fg3_pct_widget.value,
            ast_widget.value,
            reb_widget.value
        ]])

        # Scale the input without feature names
        input_scaled = scaler.transform(input_data)

        # Make predictions for both teams
        predicted_score_team = model.predict(input_scaled)[0]

        # Swap the encoded values to predict the opponent's score
        input_data_opponent = np.array([[
            opponent_encoded,
            team_encoded,  # Switch the team and opponent
            fg_pct_widget.value,
            ft_pct_widget.value,
            fg3_pct_widget.value,
            ast_widget.value,
            reb_widget.value
        ]])

        input_scaled_opponent = scaler.transform(input_data_opponent)
        predicted_score_opponent = model.predict(input_scaled_opponent)[0]

        # Output the prediction
        print(f"🏀 Predicted Score for {team_widget.value} vs {opponent_widget.value}:")
        print(f"👉 {int(predicted_score_team)} points for {team_widget.value}")
        print(f"👉 {int(predicted_score_opponent)} points for {opponent_widget.value}")

# Bind prediction function to button click
predict_button.on_click(predict_score)

# Layout grouping with visual appeal
team_box = widgets.HBox([team_widget, opponent_widget])
stats_box = widgets.VBox([fg_pct_widget, ft_pct_widget, fg3_pct_widget,
                          ast_widget, reb_widget])

# Show all widgets with a styled layout
display(team_box, stats_box, predict_button, output)

Button(button_style='success', description='Predict Score', layout=Layout(height='40px', width='50%'), style=B…

Output()